In [1]:
!pip install h2o

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import json
import os
import re

from datetime import datetime

import h2o
from h2o.automl import H2OAutoML

In [3]:
j = open('param_h2o.json', 'r').read()
param = json.loads(j)
param

{'sourcePath': 'https://s3model.blob.core.windows.net/prodata/merged_proc_data.csv',
 'savingPath': 'https://s3model.blob.core.windows.net/modeldata/',
 'target': 'average_ticket',
 'excludeAlgos': 'DeepLearning',
 'algos': ['DRF', 'GLM', 'XGBoost', 'GBM', 'DeepLearning', 'StackedEnsemble'],
 'maxModels': '10',
 'seed': '6',
 'maxTrainingTime': '105',
 'classification': 'False'}

In [4]:
source_path = param['sourcePath']
saving_path = param['savingPath']
target = param['target']
exclude_algos = [param['excludeAlgos']]
max_models = int(param['maxModels'])
seed = int(param['seed'])
max_training_time = int(param['maxTrainingTime'])
classification = param['classification']

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312"; OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~20.04-b07); OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Starting server from /home/hobbes/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpaw3kmytv
  JVM stdout: /tmp/tmpaw3kmytv/h2o_hobbes_started_from_python.out
  JVM stderr: /tmp/tmpaw3kmytv/h2o_hobbes_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_hobbes_5iva5i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.694 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [6]:
df = h2o.import_file(path=source_path)
train, test = df.split_frame(ratios=[0.8], seed = seed)

aml = H2OAutoML(
    max_models = max_models, 
    max_runtime_secs = max_training_time, 
    exclude_algos = exclude_algos, 
    balance_classes=True, 
    seed = seed
)

aml.train(training_frame = train, y = target)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
15:42:42.478: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

██████████████████
15:43:13.537: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

██████████████████████████████████████████
15:44:23.741: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

███| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220121_154242


Model Summary: 


,,number_of_trees
0,,17.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 10.366098997374197
RMSE: 3.21964268163009
MAE: 1.757351379042358
RMSLE: 0.059144805734265345
Mean Residual Deviance: 10.366098997374197

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 15.145961857353797
RMSE: 3.8917813218825383
MAE: 2.3947651488264166
RMSLE: 0.07231807690689965
Mean Residual Deviance: 15.145961857353797

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,2.394751,0.315786,2.726640,2.671842,2.333775,2.291162,1.950338
1,mean_residual_deviance,15.145898,2.430011,18.531109,16.912865,13.502837,13.692509,13.090173
2,mse,15.145898,2.430011,18.531109,16.912865,13.502837,13.692509,13.090173
3,r2,0.957413,0.006977,0.948167,0.951714,0.962015,0.961556,0.963612
4,residual_deviance,15.145898,2.430011,18.531109,16.912865,13.502837,13.692509,13.090173
5,rmse,3.882059,0.307233,4.304778,4.112526,3.674621,3.700339,3.618034
6,rmsle,0.072196,0.004700,0.078544,0.075864,0.069271,0.069326,0.067974



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-01-21 16:43:07,24.991 sec,0.0,58.508976,55.384662,3423.300280
1,,2022-01-21 16:43:09,26.887 sec,5.0,11.295843,9.847778,127.596065
2,,2022-01-21 16:43:11,28.562 sec,10.0,4.105814,2.673750,16.857705
3,,2022-01-21 16:43:12,30.106 sec,15.0,3.324741,1.814013,11.053906
4,,2022-01-21 16:43:13,30.742 sec,17.0,3.219643,1.757351,10.366099



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,price_range,1.074409e+07,1.000000,0.835658
1,minimum_order_value,4.221658e+05,0.039293,0.032835
2,order_total_amount,2.780739e+05,0.025882,0.021628
3,delivery_time,1.562586e+05,0.014544,0.012154
4,merchant_zip_code,1.427980e+05,0.013291,0.011107
5,enabled.False,1.289582e+05,0.012003,0.010030
6,delivery_address_longitude,1.243419e+05,0.011573,0.009671
7,takeout_time,8.698420e+04,0.008096,0.006765
8,enabled.True,7.839837e+04,0.007297,0.006098
9,delivery_address_latitude,7.462602e+04,0.006946,0.005804



See the whole table with table.as_data_frame()


In [7]:
best_model = aml.get_best_model()

model_info = best_model.model_performance(test)

In [8]:
local_path = os.getcwd()+'/'

In [9]:
model_path_1 = h2o.save_model(model=best_model,path=local_path, force=True)

In [10]:
best_model = model_path_1.split('/')[int(len(model_path_1.split('/'))-1)]

model_id = best_model.split('_')[-2]+best_model.split('_')[-1]
model_name = best_model.split('_')[0]+'_'+model_id
model_date = best_model.split('_')[-2]

In [11]:
test_name = 'test_' + model_id
train_name = 'train_' + model_id

In [12]:
test.as_data_frame().convert_dtypes().to_pickle(test_name + '.csv')
train.as_data_frame().convert_dtypes().to_pickle(train_name + '.csv')

In [13]:
model_path = saving_path + model_name

try:
    aic = model_info.aic()
except:
    aic = str(0)
    
info1 = {
        "id" : str(model_id),
        "name" : str(model_name),
        "date" : datetime. strptime(model_date, '%Y%m%d'),
        "h2o_version" : str(h2o.__version__),
        "seed" : str(seed),
        "target" : str(target),
        "train_path" : str(saving_path + test_name+ '.csv'),
        "test_path" : str(saving_path + train_name+ '.csv'),
        "model_path" : str(saving_path + model_name),
        "max_runtime" : str(aml.max_runtime_secs),
        "classification" : str(classification),
        "columns" : str(list(aml.varimp().index)),
        "varimp" : str(aml.varimp().to_json())
}


if classification == 'False':
    info2 = {
        "metrics":{
            "mse":str(model_info.mse()),
            "rmse":str(model_info.rmse()),
            "mae":str(model_info.mae()),
            "rmsle":str(model_info.rmsle()),
            "r2":str(model_info.r2()),
            "mean_resid_deviance":str(model_info.mean_residual_deviance()),
            "null_degrees_of_freedom":str(model_info.null_degrees_of_freedom()),
            "resid_degrees_of_freedom":str(model_info.residual_degrees_of_freedom()),
            "null_deviance":str(model_info.null_deviance()),
            "residual_deviance":str(model_info.residual_deviance()),
            "aic":aic,
        }
    }
    
info = info1.copy()
info.update(info2)

In [14]:
info_new = pd.DataFrame([info])

info_path = local_path + 'info.csv'

try:
    info_old = pd.read_csv(info_path)
    if info_old.columns[0] == 'Unnamed: 0':
        info_old = info_old.drop(info_old.columns[0], axis=1)
    
    info_save = pd.concat([info_old, info_new])
    info_save.to_csv(info_path, index=False)
except:
    info_new.to_csv(info_path, index=False)

In [15]:
os.rename(model_path_1, local_path + model_name)

In [20]:
# h2o.shutdown(prompt = True)